# Buy and sell coins based on Binance_live_crypto.ipynb
### Thanks to Algovibes YouTube channel

In [1]:
import asyncio
from binance import AsyncClient, BinanceSocketManager
import pandas as pd
import datetime as dt
from binance.client import Client
from sqlalchemy import create_engine

api_key = '0RKDJcPdqgKzMy4I2j2dXusji6RPSkfU8hJMeP8ssaQTDv64qGCLmvnCEiejw09u'
api_secret = 'nJIIKm9e77fJ0EqbFYDgwgbfUyglywMMjbuhDOCbVd1DCrlwdReBmtNm6PgqnCxK'
engine = create_engine('sqlite:///CryptoDB.db')
client = Client(api_key, api_secret)

In [2]:
symbols = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', engine).name.tolist()
def qry(symbol, lookback:int):
    now = dt.datetime.now() - dt.timedelta(hours=1)	# binance time
    before = now - dt.timedelta(minutes=lookback)
    qry_str = f"""SELECT * FROM '{symbol}' WHERE time >= '{before}'"""
    return pd.read_sql(qry_str, engine)

In [3]:
rets=[]
for symbol in symbols:
    prices = qry(symbol,3).Price
    cumret = (prices.pct_change()+1).prod()-1
    rets.append(cumret)

In [4]:
if len(rets)>0:
    top_coin = symbols[rets.index(max(rets))]
    print(top_coin, max(rets))
else:
    top_coin=None
    print('No coins to buy')

TFUELUSDT 0.02744201241424382


In [5]:
investment_amount = float(300)
MY_BALANCE = float(500)  ##### This is a temporary variable before we go sharp
 
def get_my_balance():
    global MY_BALANCE
    if False:
        free_usd = [i for i in client.get_account()['balances'] if i['asset']=='USDT'][0]['free']
    
    # temporary override free_usd
    free_usd =  MY_BALANCE
    
    return free_usd

In [6]:
def create_buy_order(symbol, quantity,price):
    global MY_BALANCE
    if False: 
        order = client.order_market_buy(symbol=symbol, side='BUY',type='MARKET', quantity=quantity)
    
    # temporary until we go sharp
    priset=quantity*price
    order = {'fills':[{'price':price},{'price':price} ]}
    MY_BALANCE += priset
    return order


In [7]:

info=client.get_symbol_info(symbol=top_coin)
LotSize = float([i for i in info['filters'] if i['filterType']=='LOT_SIZE'][0]['stepSize'])
price = float(client.get_symbol_ticker(symbol=top_coin)['price'])
decimals = len(str(LotSize).split('.')[1])
buy_quantity = round(investment_amount/price,decimals)

free_usd = get_my_balance()
if float(free_usd) > investment_amount:
    order=create_buy_order(top_coin,buy_quantity,price)
    buyprice = float(order['fills'][0]['price'])
    print(f'Köpte {buy_quantity} {top_coin} för {buyprice}. Totalt: {buyprice*buy_quantity}')
else:
    buyprice=None
    print('You have not enough USDT to buy the coin', top_coin, free_usd, investment_amount)
    quit()

TFUELUSDT 952.7 0.0


In [8]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns=['symbol', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

def create_sell_order(symbol, quantity,price):
    global MY_BALANCE
    if False:
        order = client.create_order(symbol=symbol, side='SELL', type='MARKET', quantity=quantity)
    
    # temporary until we go sharp
    priset=price*quantity
    order = {'fills':[{'price':price},{'price':price} ]}
    MY_BALANCE -= priset
    return order


In [9]:
async def main(coin):
    # client = await AsyncClient.create()
    # print('start main')
    # set start time
    start_time = dt.datetime.now()
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket(coin)
    async with ts as tscm:
        # print('start trade loop')
        while True:
            res = await tscm.recv()
            if res:
                # set elapsed time since start
                elapsed_time = dt.datetime.now() - start_time
                frame = createframe(res)
                if elapsed_time.seconds > 60:
                    start_time = dt.datetime.now()
                    print(frame.Price[0])
                if frame.Price[0] < buyprice*0.97 or frame.Price[0] > buyprice*1.02:
                    order = create_sell_order(coin, buy_quantity,frame.Price[0])
                    print(f'Sålde {buy_quantity} {top_coin} för {frame.Price[0]}. Totalt: {buy_quantity*frame.Price[0]}' )
                    print('\n',order)
                    # loop.stop()
                    break
    
    # await client.close_connection()
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(top_coin))


